In [1]:
# pip install dlib
# pip install cmake
# pip install opencv-python face_recognition
# pip install face-recognition
# pip install opencv-contrib-python
# pip install --upgrade pip
# pip install tensorflow
# pip install tensorflow keras opencv-python-headless

In [2]:
# pip install --upgrade opencv-python opencv-python-headless

In [3]:
import cv2
import os
import pandas as pd
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

C:\Users\diana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# Load model
# model = load_model('fruits-cnn-ad.h5')
model = load_model('fruits-cnn-ad_AllFruits.h5')

C:\Users\diana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# Print the model summary to check the input shape
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 98, 98, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 47, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 21, 21, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 250)            │     3,200,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 28)             │         7,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,299,952 (12.59 MB)

 Trainable params: 3,299,950 (12.59 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [6]:
# Mapeo de índices a etiquetas de clases
class_labels = {
    0: 'Apple Healthy', 1: 'Apple Rotten', 2: 'Banana Healthy', 3: 'Banana Rotten',
    4: 'Bellpepper Healthy', 5: 'Bellpepper Rotten', 6: 'Carrot Healthy', 7: 'Carrot Rotten',
    8: 'Cucumber Healthy', 9: 'Cucumber Rotten', 10: 'Grape Healthy', 11: 'Grape Rotten',
    12: 'Guava Healthy', 13: 'Guava Rotten', 14: 'Jujube Healthy', 15: 'Jujube Rotten',
    16: 'Mango Healthy', 17: 'Mango Rotten', 18: 'Orange Healthy', 19: 'Orange Rotten',
    20: 'Pomegranate Healthy', 21: 'Pomegranate Rotten', 22: 'Potato Healthy', 23: 'Potato Rotten',
    24: 'Strawberry Healthy', 25: 'Strawberry Rotten', 26: 'Tomato Healthy', 27: 'Tomato Rotten'
}

In [7]:
# DataFrame para almacenar los resultados
results_df = pd.DataFrame(columns=['file_name'])

In [8]:
# Initialize the camera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Cannot open the camera")
    exit()

In [9]:
def preprocess_image(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized_frame = cv2.resize(gray_frame, (98, 98))  # Ensure this matches the input size of your model
    image_array = img_to_array(resized_frame)
    image_array = np.expand_dims(image_array, axis=-1)  # Shape (98, 98, 1)
    image_array = np.expand_dims(image_array, axis=0)  # Shape (1, 98, 98, 1)
    image_array = image_array / 255.0  # Normalize the image
    return image_array

In [10]:
def predict(image_array):
    preds = model.predict(image_array)
    label = class_labels[np.argmax(preds)]
    prob = np.max(preds)
    return label, prob

In [11]:
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Cannot receive frame (stream end?). Exiting ...")
        break
    frame = cv2.flip(frame, 1)

    # Preprocess the frame
    image_array = preprocess_image(frame)

    # Make predictions
    try:
        label, prob = predict(image_array)
        label_text = f"{label}: {prob:.2f}"
        cv2.putText(frame, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)  # Green text

        # Print or store the file name
        print("Prediction:", label_text)
        new_row = pd.DataFrame({'file_name': [label_text]})
        results_df = pd.concat([results_df, new_row], ignore_index=True)
    except Exception as e:
        print(f"Error making prediction: {e}")

    # Show the image
    cv2.imshow('Fruit Detection', frame)

    # Save the frame with prediction
    # cv2.imwrite(os.path.join(save_dir, f'frame_{frame_count}.jpg'), frame)
    # frame_count += 1

    # End capture with 'ESC' key
    if cv2.waitKey(1) & 0xFF == 27:
        break

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
Prediction: Tomato Healthy: 0.41
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Prediction: Apple Healthy: 0.23
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Prediction: Apple Healthy: 0.21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Prediction: Apple Healthy: 0.14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Prediction: Apple Healthy: 0.23
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Prediction: Apple Healthy: 0.26
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Prediction: Mango Healthy: 0.16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Prediction: Mango Healthy: 0.21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediction: Apple Healthy: 0.19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Prediction: Mango Healthy: 0.27
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Prediction: Apple Healthy: 0.27
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Prediction: Mango Healthy: 0.34
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction: Mango Healthy: 0.33
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Prediction: Apple Healthy: 0.23
1/1 ━━━━━━━━━━━━━━

In [12]:
# Release resources
cap.release()
cv2.destroyAllWindows()

# Display the results DataFrame
results_df

,file_name
0,Tomato Healthy: 0.41
1,Apple Healthy: 0.23
2,Apple Healthy: 0.21
3,Apple Healthy: 0.14
4,Apple Healthy: 0.23
...,...
289,Tomato Healthy: 0.33
290,Tomato Rotten: 0.38
291,Tomato Rotten: 0.35
292,Tomato Rotten: 0.34
